# Neural search demo - initial indexing

Code in this notebook shows how to prepare data for indexing in a vector search engine.

It contains the following steps:

* Downloading text data which we want to search
* Initialization of pre-trained text vectorization models (with SentenceTransformer)
* Converting text data into vectors and saving it.

In [1]:
# We will use startup descriptions in this neural search demo
# Data source: https://startups-list.com/
# It contains name, short descrition, logo and location of startups.
# !wget https://storage.googleapis.com/generall-shared-data/startups_demo.json

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
# This code will download and create a pre-trained sentence encoder

# distilbert - is a distilated (lightweight) version of BERT model
# stsb - denotes that the model was trained for Semantic Textual Similarity
# Full list of available models could be found here https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device="cuda")

In [5]:
df = pd.read_json('./startups_demo.json', lines=True)

In [8]:
# Here we ancode all startup descriptions
# We do encoding in batches, as this reduces overhead costs and significantly speeds up the process
vectors = []
batch_size = 64
batch = []
for row in tqdm(df.itertuples()):
  description = row.alt + ". " + row.description
  batch.append(description)
  if len(batch) >= batch_size:
    vectors.append(model.encode(batch))  # Text -> vector encoding happens here
    batch = []

if len(batch) > 0:
  vectors.append(model.encode(batch))
  batch = []

vectors = np.concatenate(vectors)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [9]:
# Now we have all our descriptions converted into vectors.
# We have 40474 vectors of 768 dimentions. The output layer of the model has this dimension
vectors.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# You can download this saved vectors and continue with rest part of the tutorial.
np.save('vectors.npy', vectors, allow_pickle=False)

## Optional part - make a test query

Let's just make sure, that our vectors are correctly converted and make sense.

For this we manually search for a closest vectors of a random sample.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Take a random description as a query
sample_query = df.iloc[12345].description
print(sample_query)

Dental transparency provider
Deenty is a dentistry marketplace. Find your next dentist in an easy, quick and transparent way. Find reviews, sales prices and education about your next dental treatment.


In [ ]:
query_vector = model.encode(sample_query)  # Convert query description into a vector.

In [ ]:
scores = cosine_similarity([query_vector], vectors)[0]  # Look for the most similar vectors, manually score all vectors
top_scores_ids = np.argsort(scores)[-5:][::-1]  # Select top-5 with vectors the largest scores

In [ ]:
# Check if result similar to the query
for top_id in top_scores_ids:
  print(df.iloc[top_id].description)
  print("-----")

Dental transparency provider
Deenty is a dentistry marketplace. Find your next dentist in an easy, quick and transparent way. Find reviews, sales prices and education about your next dental treatment.
-----
Dental management made easy
Dentalink is a web based monthly fee software, for the management of dental clinics and practices. With it, you can manage all the resources within, from appointment, notifications, email marketing to performance reports, like cash flow, quotes uptake rate, and ...
-----
A marketplace for healthcare supplies.
Supply Clinic is an online marketplace for dental and healthcare supplies. We aggregate the low-touch end of the supply market on a single platform, bringing transparency to an otherwise opaque and inefficient market. Clinicians can browse our site, compare product ...
-----
Mobile Apps for Dental Care
MoodUp - all dental stuff in one app
We create mobile apps for developing good healthy habits in children. We connect the app with both their dentists